# BPipe permissioning

Every market data feed is subject to strict licensing rules set by exchanges, which define who can access what information and how it can be used. Without fine-grained entitlements, providers risk over-distribution of data, leading to compliance violations, financial penalties, and even loss of licensing rights. By enforcing entitlements, Bloomberg can guarantee that clients receive the right data, exchanges are properly compensated, and the entire ecosystem maintains trust and regulatory alignment.

## Reading our BPipe table
Without any permissioning enforced, we read all bpipe table as-is regardless on user permissioning.
This is just for testing purpose

In [0]:
SELECT * FROM market_data.providers.bloomberg_bpipe
LIMIT 20

SECURITY,MKTDATA_EVENT_TYPE,MKTDATA_EVENT_SUBTYPE,EID,LAST_PRICE,IS_DELAYED_STREAM,TRADE_UPDATE_STAMP_RT,processed_timestamp
GBP BGN Curncy,SUMMARY,INITPAINT,33828,1.3342,false,null,2025-09-25T20:22:42.749Z
AB020406 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
AB030510 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
AB050710 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
5RIZFU2 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
EUR BGN Curncy,SUMMARY,INITPAINT,33828,1.1665,false,null,2025-09-25T20:22:42.749Z
AB021030 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
BGN Curncy,SUMMARY,INITPAINT,33828,1.677,false,null,2025-09-25T20:22:42.749Z
AB020310 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
JPYEUR BGN Curncy,SUMMARY,INITPAINT,33828,0.57244,false,null,2025-09-25T20:22:42.749Z


## Reading our entitlements tables
This follows a SCD type 2 model so that entitlements are revoked but not dropped and ensure full audit.
Furthermore, can allow back processing / reply with as-of permissions (if required)

In [0]:
SELECT * FROM market_data_entitlements.public.bpipe;

id,entitlement_id,principal_id,is_granted,version,effective_from,effective_to,is_current
2,38736,antoine.amend+isv@databricks.com,true,1,2025-09-25T19:08:29.811Z,null,true
3,39489,antoine.amend+isv@databricks.com,true,1,2025-09-25T19:08:43.624Z,null,true
1,33828,antoine.amend+isv@databricks.com,true,1,2025-09-25T19:07:30.898Z,2025-09-25T20:06:48.665Z,false
4,33828,antoine.amend+isv@databricks.com,false,2,2025-09-25T20:06:48.665Z,2025-09-25T20:07:22.826Z,false
5,33828,antoine.amend+isv@databricks.com,true,3,2025-09-25T20:07:22.826Z,null,true


## Create a permissioning function
We create a simple UC function atop of our entitlement table to test user permissioning. 
Given a EID, we check user ownership and entitlement status.

In [0]:
CREATE OR REPLACE FUNCTION market_data.providers.is_bpipe_eid_member(
  eid STRING COMMENT 'The bloomberg entitlement ID we want to test user permissioning against'
  )
RETURNS BOOLEAN
COMMENT 'Test if user has permission to a given bloomberg entitlement Id'
RETURN EXISTS (
  SELECT * FROM market_data_entitlements.public.bpipe
  WHERE principal_id = current_user
  AND entitlement_id = eid
  AND is_current
  AND is_granted
);

Let us play with our UI to grant or revoke specific entitlements. 
We should see permissioning being reflected here right away since we do not depend on any ETL

In [0]:
SELECT market_data.providers.is_bpipe_eid_member('33828') AS IS_GRANTED;

IS_GRANTED
true


## Reading our BPipe table
With our permissioning function registered, we can use entitlement as ways to filter out records we are not entitled to see.
This is just for testing purpose and will be enforced later.

In [0]:
SELECT * FROM market_data.providers.bloomberg_bpipe
WHERE market_data.providers.is_bpipe_eid_member(EID)
LIMIT 20;

SECURITY,MKTDATA_EVENT_TYPE,MKTDATA_EVENT_SUBTYPE,EID,LAST_PRICE,IS_DELAYED_STREAM,TRADE_UPDATE_STAMP_RT,processed_timestamp
GBP BGN Curncy,SUMMARY,INITPAINT,33828,1.3342,false,null,2025-09-25T20:22:42.749Z
AB020406 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
AB030510 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
AB050710 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
5RIZFU2 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
EUR BGN Curncy,SUMMARY,INITPAINT,33828,1.1665,false,null,2025-09-25T20:22:42.749Z
AB021030 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
BGN Curncy,SUMMARY,INITPAINT,33828,1.677,false,null,2025-09-25T20:22:42.749Z
AB020310 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
JPYEUR BGN Curncy,SUMMARY,INITPAINT,33828,0.57244,false,null,2025-09-25T20:22:42.749Z


## Enforcing entitlement policies
We can enforce that function by creating a row filter in our registered bpipe table

<img src='bpipe_row_filter_enabled.png'>

## Testing our entitlement policy
And voila, with our entitlement policy being enforced, we can restrict access to specific records based on EID entitlements. This policy table is always one as available on Lakebase, and guarantees audit & traceability given its SCD type 2 schema

In [0]:
SELECT * FROM market_data.providers.bloomberg_bpipe
LIMIT 20;

SECURITY,MKTDATA_EVENT_TYPE,MKTDATA_EVENT_SUBTYPE,EID,LAST_PRICE,IS_DELAYED_STREAM,TRADE_UPDATE_STAMP_RT,processed_timestamp
GBP BGN Curncy,SUMMARY,INITPAINT,33828,1.3342,false,null,2025-09-25T20:22:42.749Z
AB020406 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
AB030510 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
AB050710 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
5RIZFU2 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
EUR BGN Curncy,SUMMARY,INITPAINT,33828,1.1665,false,null,2025-09-25T20:22:42.749Z
AB021030 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
BGN Curncy,SUMMARY,INITPAINT,33828,1.677,false,null,2025-09-25T20:22:42.749Z
AB020310 BGN Curncy,SUMMARY,INITPAINT,33828,null,false,null,2025-09-25T20:22:42.749Z
JPYEUR BGN Curncy,SUMMARY,INITPAINT,33828,0.57244,false,null,2025-09-25T20:22:42.749Z
